
# Business Intelligence ETL Using Pandas 

### By Welemhret Welay Baraki

### Import the necessary python packages

In [97]:
#Import the necessary Python Libraries 
import pandas as pd 
import numpy as np 

### Import the dataset for the years 1989 and 1999 

In [98]:
#Import the datasets of Flight information for the Years 1989 and 1999
Data1989=pd.read_csv("BI datasets/1989.csv")
Data1999=pd.read_csv("BI datasets/1999.csv")
#Import the US Airtports dataset to extract the states that the airport belongs
UsAirpots=pd.read_csv("BI datasets/us-airports.csv")
#Importing WorldAirports
WorldAirPorts=pd.read_csv("BI datasets/airports_iata_codes.csv")

In [99]:
#Explore the size of the datasets
Data1989.shape, Data1999.shape,UsAirpots.shape,WorldAirPorts.shape

((5041200, 29), (5527884, 29), (29008, 23), (8738, 8))

### Adding Airport Names based on the IATA Code from Airports IATA codes Datasets

In [100]:
#Selecting the Airport IATA Code and their Corresponding Local Names
WorldAirPorts=WorldAirPorts[["iata_code","name"]]
#Change the column names to ["Origin","AirPortName"]
WorldAirPorts.columns=["Origin","AirPortName"]

### Adding Airport Name based on the Origin column


In [102]:
Data1989=pd.merge(Data1989,WorldAirPorts,on=['Origin'], how='left')
Data1999=pd.merge(Data1999,WorldAirPorts,on=['Origin'], how='left')

In [103]:
Data1999.tail()

,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,...,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay,AirPortName
5527879,1999,12,16,4,1910.0,1910,2112.0,2117,AA,1359,...,12,0,NaN,0,NaN,NaN,NaN,NaN,NaN,Nashville International Airport
5527880,1999,12,17,5,1909.0,1910,2111.0,2117,AA,1359,...,15,0,NaN,0,NaN,NaN,NaN,NaN,NaN,Nashville International Airport
5527881,1999,12,18,6,1907.0,1910,2103.0,2117,AA,1359,...,9,0,NaN,0,NaN,NaN,NaN,NaN,NaN,Nashville International Airport
5527882,1999,12,19,7,1907.0,1910,2114.0,2117,AA,1359,...,13,0,NaN,0,NaN,NaN,NaN,NaN,NaN,Nashville International Airport
5527883,1999,12,20,1,1905.0,1910,2127.0,2117,AA,1359,...,8,0,NaN,0,NaN,NaN,NaN,NaN,NaN,Nashville International Airport


### Adding AirPort State/Region in the US based on the IATA Code from the UsAirpots Datasets

In [104]:
#Extract the IATA code of the Airports and States/Region Name/ location of the Airport
#Reset index and drop duplicates to filter list of airports and their regions
AirportRegion=UsAirpots[["iata_code","region_name"]].reset_index(drop=True).drop_duplicates(subset=["iata_code","region_name"], keep='last')

In [105]:
#Drop Null Values 
AirportRegion=AirportRegion.dropna()

In [106]:
# Rename the iata_code column of the dataframe with "Origin" 
# to be able to merge with the datasets of 1989 and 1999
AirportRegion.rename(columns={"iata_code":"Origin"},inplace=True)

In [107]:
#Airport Region Columns
#Add the missing region names for the following airports
MisseDetails={
    "Origin":['PFN', 'STT', 'STX', 'UCA', 'SJU', 'GUM', 'YAP', 'SPN','ROP'],
    "region_name":['Florida','US Virgin Islands','US Virgin Islands','New York',
                   'Puerto Rico','Guam','Yap Islands','Mariana Islands','Mariana Islands']}
Missed=pd.DataFrame(MisseDetails)
Missed
AirportRegion=pd.concat([AirportRegion, Missed])

In [108]:
#Add the States based on the Origin Airport to the data of the year 1989
Data1989=pd.merge(Data1989,AirportRegion,on=['Origin'], how='left')
#Add the States based on the Origin Airport to the data of 1999
Data1999=pd.merge(Data1999,AirportRegion,on=['Origin'], how='left')

In [109]:
# Check if the there are null values in the region name in 1999
Data1989[Data1989["region_name"].isna() ]["Origin"].unique()
# Check if the there are null values in the region name in 1999
Data1999[Data1999["region_name"].isna() ]["Origin"].unique()

array([], dtype=object)

### Adding the Air Line Name based on the Unique Carrier IATA Codes

In [110]:
# List the 13 Unique Carriers /Air Lines/ 
Data1989['UniqueCarrier'].unique()

array(['UA', 'US', 'WN', 'NW', 'PA (1)', 'PI', 'TW', 'DL', 'EA', 'HP',
       'AA', 'AS', 'CO'], dtype=object)

In [111]:
Data1999['UniqueCarrier'].unique()

array(['US', 'UA', 'AA', 'DL', 'TW', 'HP', 'WN', 'CO', 'NW', 'AS'],
      dtype=object)

In [112]:
USAirlines={
    "UniqueCarrier":['UA', 'US', 'WN', 'NW', 'PA', 'PI', 'TW', 'DL', 'EA', 'HP', 'AA', 'AS', 'CO'],
    "AirLines":["United Air Lines Inc.","US Airways Inc.","Southwest Airlines Co.","Northwest Airlines Inc.",
               "Florida Coastal Airlines","Piedmont Aviation Inc.","Trans World Airways LLC", "Delta Air Lines Inc.",
               "Eastern Air Lines Inc.","America West Airlines Inc.","American Airlines Inc.",
               "Alaska Airlines Inc.","Continental Air Lines Inc."
               
              ]}
USAirlines=pd.DataFrame(USAirlines)
USAirlines

,UniqueCarrier,AirLines
0,UA,United Air Lines Inc.
1,US,US Airways Inc.
2,WN,Southwest Airlines Co.
3,NW,Northwest Airlines Inc.
4,PA,Florida Coastal Airlines
5,PI,Piedmont Aviation Inc.
6,TW,Trans World Airways LLC
7,DL,Delta Air Lines Inc.
8,EA,Eastern Air Lines Inc.
9,HP,America West Airlines Inc.


In [113]:
# Add AirLines column that identifies The Unique Carriers code with Human readable and easily understandable
Data1989=pd.merge(Data1989,USAirlines,on=['UniqueCarrier'], how='left')
Data1999=pd.merge(Data1999,USAirlines,on=['UniqueCarrier'], how='left')

In [114]:
Data1999

,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,...,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay,AirPortName,region_name,AirLines
0,1999,1,27,3,1906.0,1908,2024.0,2005,US,1244,...,NaN,0,NaN,NaN,NaN,NaN,NaN,Richmond International Airport,Virginia,US Airways Inc.
1,1999,1,28,4,2016.0,1908,2126.0,2005,US,1244,...,NaN,0,NaN,NaN,NaN,NaN,NaN,Richmond International Airport,Virginia,US Airways Inc.
2,1999,1,29,5,1907.0,1908,2000.0,2005,US,1244,...,NaN,0,NaN,NaN,NaN,NaN,NaN,Richmond International Airport,Virginia,US Airways Inc.
3,1999,1,31,7,1932.0,1908,2031.0,2005,US,1244,...,NaN,0,NaN,NaN,NaN,NaN,NaN,Richmond International Airport,Virginia,US Airways Inc.
4,1999,1,1,5,1601.0,1535,1707.0,1645,US,297,...,NaN,0,NaN,NaN,NaN,NaN,NaN,Greater Rochester International Airport,New York,US Airways Inc.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5527879,1999,12,16,4,1910.0,1910,2112.0,2117,AA,1359,...,NaN,0,NaN,NaN,NaN,NaN,NaN,Nashville International Airport,Tennessee,American Airlines Inc.
5527880,1999,12,17,5,1909.0,1910,2111.0,2117,AA,1359,...,NaN,0,NaN,NaN,NaN,NaN,NaN,Nashville International Airport,Tennessee,American Airlines Inc.
5527881,1999,12,18,6,1907.0,1910,2103.0,2117,AA,1359,...,NaN,0,NaN,NaN,NaN,NaN,NaN,Nashville International Airport,Tennessee,American Airlines Inc.
5527882,1999,12,19,7,1907.0,1910,2114.0,2117,AA,1359,...,NaN,0,NaN,NaN,NaN,NaN,NaN,Nashville International Airport,Tennessee,American Airlines Inc.


### Exploring the data of the Years 1989 and 1999

In [115]:
#Descriptive Statistics of the year 1989
Data1989.describe().T

,count,mean,std,min,25%,50%,75%,max
Year,5041200.0,1989.000000,0.000000,1989.0,1989.0,1989.0,1989.0,1989.0
Month,5041200.0,6.518804,3.469415,1.0,3.0,7.0,10.0,12.0
DayofMonth,5041200.0,15.708359,8.781560,1.0,8.0,16.0,23.0,31.0
DayOfWeek,5041200.0,3.956539,1.993352,1.0,2.0,4.0,6.0,7.0
DepTime,4967035.0,1364.867275,478.128976,1.0,943.0,1341.0,1750.0,2400.0
CRSDepTime,5041200.0,1356.045163,471.424060,1.0,935.0,1335.0,1743.0,2400.0
ArrTime,4952196.0,1497.377673,497.331375,1.0,1119.0,1523.0,1921.0,2400.0
CRSArrTime,5041200.0,1496.717477,485.221218,1.0,1118.0,1521.0,1916.0,2400.0
FlightNum,5041200.0,773.888815,582.273113,1.0,325.0,640.0,1103.0,9368.0
TailNum,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [116]:
#Descriptive Statistics of the year 1999
Data1999.describe().T

,count,mean,std,min,25%,50%,75%,max
Year,5527884.0,1999.000000,0.000000,1999.0,1999.0,1999.0,1999.0,1999.0
Month,5527884.0,6.564378,3.434520,1.0,4.0,7.0,10.0,12.0
DayofMonth,5527884.0,15.706164,8.776228,1.0,8.0,16.0,23.0,31.0
DayOfWeek,5527884.0,3.935001,1.986862,1.0,2.0,4.0,6.0,7.0
DepTime,5373573.0,1354.402954,488.349982,1.0,930.0,1338.0,1746.0,2400.0
CRSDepTime,5527884.0,1292.790574,538.616370,0.0,900.0,1315.0,1729.0,2400.0
ArrTime,5360018.0,1490.694977,518.304143,1.0,1110.0,1523.0,1925.0,2400.0
CRSArrTime,5527884.0,1437.879420,571.806206,0.0,1049.0,1502.0,1913.0,2400.0
FlightNum,5527884.0,1089.734731,748.170985,1.0,463.0,982.0,1648.0,6537.0
ActualElapsedTime,5360018.0,126.505774,69.920844,-344.0,74.0,108.0,159.0,1771.0


In [117]:
#Total count of records of the years 1989 and 1999
#Over 10 Million records are available in the datasets
Data1989.shape[0] + Data1999.shape[0] 

10569084

In [118]:
#Concatenate the null values counts of each columns/attributes for better understanding 
IsNull= pd.concat([Data1989.isnull().sum(),Data1999.isna().sum()], axis=1)
#Rename the column names [0,1] with the years [1989,1999]
IsNull.columns=['1989 GC','1999 GC']

In [119]:
IsNull

,1989 GC,1999 GC
Year,0,0
Month,0,0
DayofMonth,0,0
DayOfWeek,0,0
DepTime,74165,154311
CRSDepTime,0,0
ArrTime,89004,167866
CRSArrTime,0,0
UniqueCarrier,0,0
FlightNum,0,0


In [120]:
# Exploring the numbel of Null values in the features 
IsNull.loc[(IsNull['1989 GC']!=0) | (IsNull['1999 GC']!=0)]

,1989 GC,1999 GC
DepTime,74165,154311
ArrTime,89004,167866
TailNum,5041200,0
ActualElapsedTime,89004,167866
CRSElapsedTime,0,8992
AirTime,5041200,167866
ArrDelay,89004,167866
DepDelay,74165,154311
Distance,26988,0
TaxiIn,5041200,0


In [121]:
Data1989.shape,Data1999.shape

((5041200, 32), (5527884, 32))

In [122]:
#Drop the columns that have many null values 
# The columns to drop are: ===> CancellationCode,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
# Drop columns from the dataframe of the year 1989
Data1989.drop(["CancellationCode","CarrierDelay","WeatherDelay","NASDelay","SecurityDelay",
               "LateAircraftDelay"], 
             axis='columns',
             inplace=True)

In [123]:
# Drop columns from the dataframe of the year 1999
Data1999.drop(["CancellationCode","CarrierDelay","WeatherDelay","NASDelay","SecurityDelay",
               "LateAircraftDelay"], 
             axis='columns',
             inplace=True)

In [128]:
# Concatenating the two dataframes 
dframes=[Data1989,Data1999]
CombinedData=pd.concat(dframes)

In [129]:
#Departure, Arrival  and on-time Status
CombinedData['FlightStatus'] = ['Cancelled' if pd.isna(x) else ('On Time' if x<=0  else 'Delayed') for x in CombinedData['ArrDelay']]
CombinedData['FlightStatus'] = ['Cancelled' if pd.isna(x) else ('On Time' if x<=0  else 'Delayed') for x in CombinedData['DepDelay']]

In [130]:
CombinedData.dtypes

Year                   int64
Month                  int64
DayofMonth             int64
DayOfWeek              int64
DepTime              float64
CRSDepTime             int64
ArrTime              float64
CRSArrTime             int64
UniqueCarrier         object
FlightNum              int64
TailNum               object
ActualElapsedTime    float64
CRSElapsedTime       float64
AirTime              float64
ArrDelay             float64
DepDelay             float64
Origin                object
Dest                  object
Distance             float64
TaxiIn               float64
TaxiOut              float64
Cancelled              int64
Diverted               int64
AirPortName           object
region_name           object
AirLines              object
FlightStatus          object
dtype: object

### Export dataframe to csv file to visualize in the Tableau Desktop

In [131]:
CombinedData.to_csv("FinalCombinedDataModified.csv",index=False)

In [132]:
Reload1999and1989=pd.read_csv("FinalCombinedDataModified.csv")

C:\Users\Weldat\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [133]:
Reload1999and1989.head(3)

,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,...,Dest,Distance,TaxiIn,TaxiOut,Cancelled,Diverted,AirPortName,region_name,AirLines,FlightStatus
0,1989,1,23,1,1419.0,1230,1742.0,1552,UA,183,...,HNL,2398.0,NaN,NaN,0,0,San Francisco International Airport,California,United Air Lines Inc.,Delayed
1,1989,1,24,2,1255.0,1230,1612.0,1552,UA,183,...,HNL,2398.0,NaN,NaN,0,0,San Francisco International Airport,California,United Air Lines Inc.,Delayed
2,1989,1,25,3,1230.0,1230,1533.0,1552,UA,183,...,HNL,2398.0,NaN,NaN,0,0,San Francisco International Airport,California,United Air Lines Inc.,On Time


In [134]:
Reload1999and1989.dtypes

Year                   int64
Month                  int64
DayofMonth             int64
DayOfWeek              int64
DepTime              float64
CRSDepTime             int64
ArrTime              float64
CRSArrTime             int64
UniqueCarrier         object
FlightNum              int64
TailNum               object
ActualElapsedTime    float64
CRSElapsedTime       float64
AirTime              float64
ArrDelay             float64
DepDelay             float64
Origin                object
Dest                  object
Distance             float64
TaxiIn               float64
TaxiOut              float64
Cancelled              int64
Diverted               int64
AirPortName           object
region_name           object
AirLines              object
FlightStatus          object
dtype: object

## END of ETL Using Python Pandas